In [2]:
import numpy as np
import pandas as pd
import transformers
import torch

In [3]:
! pip install datasets transformers

In [4]:
import transformers
print(transformers.__version__)

4.31.0


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

In [7]:
from datasets import load_dataset
datasets = load_dataset('text', data_files={'train': "../data/train_data.txt", 'validation': "../data/test_data.txt"})

In [8]:
datasets['train'][10]

{'text': "customer: Hi, is this jacket dry clean only? agent: Hello, it is not dry clean only. We use high quality fabric in our materials, so you can wash and dry them in a typical washer and dryer! customer: Great! It is a little inconvenient when one is not machine washable. agent: I highly agree! Especially with everything happening right now, you may not even be able to find one that is open! agent: Anyway, is there anything else I can help you with? customer: Do they come in different colors agent: Hello, I would have to direct you to our website, as our stock is very dynamic. If you do not see the color you like, I suggest checking back at the end of the month, when we refill the stock customer: That's alright. I actually want it in red if you have it. But if not, I'll get this. agent: That sounds great! Is there anything else I can help you with? customer: this is good. thank you agent: Have a great day!"}

In [9]:
from datasets import ClassLabel
import random
from IPython.display import display,HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more"
    picks=[]
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0,len(dataset)-1)
        picks.append(pick)
        
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets['train'])

,text
0,"agent: AcmeBrands, how may I help you? customer: Yes i would like to pay my subscription fee to keep it active agent: Let me help you with that. agent: Could I have your full name or account id? customer: Joyce Wu and account id is ZXJXRIMAJ4 agent: Could I have your order id? customer: 1627431558 agent: Thank you agent: The standard subscription is $99, how much of that would you like to pay? agent: It is for the full year. customer: $99 and i would like to use my card i have on file already agent: Ok agent: Your subscription has been renewed for a year and you paid $99 using the card on file. customer: thank you agent: Can I help you with anything else? customer: that will be all agent: Have a good day."
1,"agent: Hello, can I help you with something? customer: I'm having this crazy frustrating issue with your site. Every time I add something to my order, the shopping cart doesn't update. Can you help? agent: Yes. Have you tried logging in and out yet? customer: Yeah, first thing I tried! agent: Refresh the page and try to add it again, please. customer: Well dang, refreshing fixed it! customer: Thanks a lot for that. agent: That's great! Is there anything else I can help you with? customer: Nah, all good! agent: Have a great day!"
2,"agent: Hello, how can I help you? customer: Hi customer: I'm trying to buy something but my card keeps getting rejected. agent: Okay, can you try to re-enter the card information one more time? customer: I tried customer: it didn't work agent: Since that didn't work, try to log out and log back in. It might work afterwards. customer: No, it still isn't working agent: Can you double-check the expiration date? Perhaps the card is expired. customer: Oh customer: yes, it's expired customer: I'll try another card then customer: Thanks agent: Okay, glad we found out what was wrong. Have a nice day."
3,"agent: Thank you for contacting Acme brand. May I have your name please? customer: yes i was trying to see why the website is running really slow today. It not usually like this agent: Ok, have you tried logging in and out of your account? customer: ok one sec customer: no still doing the same thing agent: Have you tried closing out all other browser tabs? customer: yes it's still slow agent: ok lets try one more thing customer: ok agent: will you try to visit another website to see if it is slow too? try facebook.com customer: okay i seems to be working better now agent: great! is there anything else I can do for you? customer: no that would be all agent: thank you for being a customer"
4,agent: Hi. How may I help you today? customer: I forgot my username agent: What's your full name? customer: Name: Crystal Minh Phone Number: (668) 114-1252 Member Level: Bronze agent: Can I have your zip code or email address? customer: 61180 agent: I located your account with us. agent: Your username is cminh1 agent: Can I help with anything else? customer: thanks agent: You agent: are welcome@ agent: Is there anything else I can do for you today? customer: no agent: Great. Have a lovely day
5,"customer: Hi. agent: Hello. How may I assist you today? customer: So I lost my phone while I was visiting my daughter customer: It's the phone I use for my two-factor authentication. Now I can't get into my account. agent: Gotcha, so sorry to hear that. What's your name? customer: I don't know what to do customer: Crystal Minh agent: Can I have your email address? customer: yes customer: it's minh_2938@aol.com agent: Thanks. agent: I am going to send you an email Crystal. It will tell you how to reset that two-factor authentication. customer: Thanks! customer: That will be really useful agent: In addition, I am going to send you an email which explains our suggestions for best security practices. agent: Do you know your username? customer: ummm.... customer: I think it's minhthegreat agent: Okay, then. Just wanted to make sure you knew it! customer: lol agent: Can I help you with anything else? customer: Can you wait

In [11]:
model_checkpoint='distilgpt2'

In [26]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token':'[PAD]'})

Using pad_token, but it is not set yet.


AssertionError: Key pad_toekn is not a special token

In [13]:
def tokenize_function(examples,tokenizer=tokenizer):
    return tokenizer(examples['text'])

In [14]:
tokenized_datasets= datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=['text'])


In [15]:
tokenized_datasets['train'][1]

{'input_ids': [25781,
  25,
  922,
  6672,
  11,
  703,
  460,
  314,
  1037,
  345,
  30,
  6491,
  25,
  655,
  2227,
  284,
  2198,
  319,
  262,
  3722,
  286,
  257,
  12929,
  5797,
  25,
  1654,
  11,
  561,
  345,
  1577,
  502,
  534,
  1336,
  1438,
  393,
  1848,
  4522,
  6491,
  25,
  47319,
  28092,
  9643,
  6491,
  25,
  49812,
  8538,
  24,
  2670,
  5797,
  25,
  3224,
  284,
  428,
  345,
  561,
  1577,
  502,
  262,
  1502,
  4522,
  290,
  3053,
  5797,
  25,
  3387,
  6491,
  25,
  9225,
  1433,
  3134,
  2414,
  1983,
  6491,
  25,
  49812,
  8538,
  24,
  2670,
  31,
  12888,
  13,
  785,
  6491,
  25,
  645,
  18572,
  5797,
  25,
  2279,
  287,
  1502,
  11,
  2582,
  314,
  481,
  7603,
  262,
  3722,
  286,
  534,
  12929,
  13,
  6491,
  25,
  1049,
  6491,
  25,
  1309,
  502,
  760,
  5797,
  25,
  632,
  318,
  3058,
  287,
  4371,
  290,
  262,
  6074,
  2446,
  351,
  543,
  340,
  318,
  852,
  13686,
  318,
  2691,
  3371,
  534,
  3884,
  2657,
  13

In [16]:
#block_size=tokenizer.model_max_length #1024
block_size=128


In [17]:

def group_texts(examples, block_size=block_size):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [18]:

lm_datasets = tokenized_datasets.map(group_texts, batched=True, batch_size=1000,  num_proc=4, )


In [19]:
tokenizer.decode(lm_datasets['train'][1]['input_ids'])

" a bronze agent: ok, was the purchase made in the last 90 days? customer: No, I bought it in November. agent: ok, unfortunately because it has been more than 90 days we cannot accept the return. Would there be anything else I can help you with? customer: What if I ask really, really nicely? agent: I can escalate to my manager if you'd like agent: I'd just need your phone number. customer: (977) 625-2661 customer: I'll look forward to hearing from them. customer: Thanks for trying to help. agent: OK, I have let my manager know"

In [20]:
#Training Configuration
# Add AutoModelForCasualLM-

from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

# Tokenizer is not defined
from transformers import Trainer, TrainingArguments


In [21]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [22]:
# HTTPError
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets['train'],
    eval_dataset=lm_datasets['validation'],
    tokenizer=tokenizer,
    )

c:\Users\dotto\asapp-2b\finetuning\distilgpt2-finetuned-wikitext2 is already a clone of https://huggingface.co/yeseok/distilgpt2-finetuned-wikitext2. Make sure you pull the latest changes with `repo.git_pull()`.


In [23]:
# HTTP Error, RepositoryNotFoundError bug
import huggingface_hub
print(huggingface_hub.__version__)

0.16.4


In [24]:
# Model check
model.__class__.__name__

'GPT2LMHeadModel'

In [25]:
# Train data
trainer.train()


c:\Users\dotto\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/5283 [00:00<?, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Using pad_token, but it is not set yet.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
#Masked language modeling
model_checkpoint = "distilroberta-base"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns='text')

c:\Users\dotto\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dotto\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-65424480-3f5c9ddc4625acc745474e15;234f40b4-141a-49df-8ca1-89af0de2404e)

Repository Not Found for url: https://huggingface.co/api/models/yeseok/distilroberta-base-finetuned-wikitext2.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [ ]:
trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

NameError: name 'trainer' is not defined

In [ ]:
#Save Fine-tuned 
trainer.save_model('../data/fine_tuned_1107')

NameError: name 'trainer' is not defined

In [ ]:
trainer.push_to_hub()